<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/experiments/experiment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Experiment 8: Finding optimal depth of U-Net

### 0. Prepare Colab, Define Constants

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#! ls
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls

/content/drive/MyDrive/MachineLearning
/content/drive/MyDrive/MachineLearning/Geospatial_ML
data_exploration  experiments	     models	   pyproject.toml    scripts
docs		  image_processing   poetry.lock   README.md
evaluation	  metrics_bug.ipynb  prepare_data  requirements.txt


In [3]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from keras.utils import Sequence
from datetime import datetime
from data_exploration.mask_stats import Mask_Stats
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers.experimental import Adamax

In [4]:
total_tiles = 11121
train_tiles = 6672
test_val_tiles = 2224
data_path = "../data_colab/256_256"
experiment = "experiment_8"
batch_size = 32
tile_size = 256
step_size = 256

### 1. Create Data Generators

In [ ]:
train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5),
                          dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256),
                          dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5),
                        dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256),
                        dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_val_tiles, 256, 256, 5),
                         dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_val_tiles, 256, 256),
                         dtype=np.uint8)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()
print()
val_stats = Mask_Stats(val_split_y)
val_stats.print_stats()
print()
test_stats = Mask_Stats(test_split_y)
test_stats.print_stats()

Shape: (6672, 256, 256)
Land pixels: 195058814  44.610 %
Valid pixels: 138904480  31.767 %
Invalid pixels: 103292898  23.623 %
Sum: 6672

Shape: (2224, 256, 256)
Land pixels: 65320265  44.816 %
Valid pixels: 46246663  31.730 %
Invalid pixels: 34185136  23.454 %
Sum: 2224

Shape: (2224, 256, 256)
Land pixels: 64786699  44.450 %
Valid pixels: 46892391  32.173 %
Invalid pixels: 34072974  23.377 %
Sum: 2224


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, mmap_x, mmap_y, batch_size):
        self.x_input = mmap_x
        self.y_mask = mmap_y
        self.batch_size = batch_size
        self.num_samples = self.x_input.shape[0]

    # returns number of batches as int
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    # returns single batch
    def __getitem__(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]

        # normalization
        batch_inputs = batch_inputs / 255
        # one-hot-encoding
        batch_masks = np.array([tf.one_hot(item, depth=3).numpy() for item in batch_masks])

        # normalization + one hot encoding
        return batch_inputs, batch_masks

    def getitem_as_img(self, index):
        batch_indices = slice(index * self.batch_size, (index + 1) * self.batch_size)
        batch_inputs = self.x_input[batch_indices]
        batch_masks = self.y_mask[batch_indices]
        # normalization + one hot encoding
        return batch_inputs, batch_masks

In [ ]:
# instanciate DataGenerators
batch_size = 32

train_generator = DataGenerator(train_split_x, train_split_y, batch_size)
val_generator = DataGenerator(val_split_x, val_split_y, batch_size)
test_generator = DataGenerator(test_split_x, test_split_y, batch_size)

print(train_generator.__len__())
print(val_generator.__len__())
print(test_generator.__len__())

209
70
70


In [ ]:
train_batch = train_generator.__getitem__(9)
val_batch = val_generator.__getitem__(3)
test_batch = test_generator.__getitem__(4)


def print_batch_shapes(batch):
    print(type(batch))
    print(batch[0].shape)
    print(batch[1].shape)
    print()


# check batch shapes
print_batch_shapes(train_batch)
print_batch_shapes(val_batch)
print_batch_shapes(test_batch)

# check normalization
print('Check normalization')
print(train_batch[1].max())
print(train_batch[1].min())

print(val_batch[1].max())
print(val_batch[1].min())

print(test_batch[1].max())
print(test_batch[1].min())

print()
# check one-hot-encoding
print('check one hot encoding')
print(train_batch[0].max())
print(train_batch[0].min())

print(val_batch[0].max())
print(val_batch[0].min())

print(test_batch[0].max())
print(test_batch[0].min())

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

<class 'tuple'>
(32, 256, 256, 5)
(32, 256, 256, 3)

Check normalization
1.0
0.0
1.0
0.0
1.0
0.0

check one hot encoding
1.0
0.0
1.0
0.0
1.0
0.0


### 3. Model training
execute with premium GPU, High RAM

In [ ]:
!nvidia-smi

Tue May 16 20:58:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 400W |    693MiB / 40960MiB |      2%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


def build_unet_model(input_shape, num_layers, num_classes=3):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Encoder
    skips = []
    filters = 16
    for _ in range(num_layers):
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        skips.append(x)
        x = layers.MaxPooling2D(2)(x)
        filters *= 2

    # Bridge
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)

    # Decoder
    skips = reversed(skips)
    filters //= 2
    for skip in skips:
        x = layers.Conv2DTranspose(filters, 2, strides=2, activation='relu', padding='same')(x)
        x = layers.Concatenate()([skip, x])
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
        filters //= 2

    # Output
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

In [ ]:
all_metrics = []

for num_layers in range(1, 8):
    count = num_layers + 8
    model_metrics = []
    model_name = f'{tile_size}_{step_size}_run_{count}'
    model_path = f'../models/{experiment}/model_{model_name}.h5'

    print(f'{count} num_layers: {num_layers} Started at: {datetime.now()}')

    # Define the mIoU metric
    mean_iou = tf.keras.metrics.OneHotMeanIoU(num_classes=3, name='mean_iou')
    invalid_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[0], name='invalid_iou')
    valid_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[1], name='valid_iou')
    land_iou = tf.keras.metrics.OneHotIoU(num_classes=3, target_class_ids=[2], name='land_iou')

    # compiling model
    model = build_unet_model((256, 256, 5), num_layers)
    model.compile(optimizer=Adamax(), loss=categorical_crossentropy,
                  metrics=[mean_iou, invalid_iou, valid_iou, land_iou, 'accuracy'])
    print(model.summary())

    # callbacks
    early_stop_loss = EarlyStopping(monitor='val_loss', mode='min', patience=15)
    early_stop_acc = EarlyStopping(monitor='val_mean_iou', mode='max', patience=15)
    checkpoint = ModelCheckpoint(model_path, monitor="val_mean_iou", mode="max", save_best_only=True, verbose=1)

    # training
    model_history = model.fit(x=train_generator, epochs=100, validation_data=val_generator,
                              callbacks=[early_stop_loss, early_stop_acc, checkpoint])

    # Save model history
    with open(f'../models/{experiment}/history_{model_name}.pkl', 'wb') as file_pi:
        pickle.dump(model_history.history, file_pi)
    print('saving history completed')

    print(f'{count} Finished at: {datetime.now()}')
    print(f'{count} Metrics')
    print(model.metrics_names)
    print('training metrics')
    train_metrics = model.evaluate(train_generator, verbose=2)
    print(train_metrics)
    print('validation metrics')
    val_metrics = model.evaluate(val_generator, verbose=2)
    print(val_metrics)
    print('test metrics')
    test_metrics = model.evaluate(test_generator, verbose=2)
    print(test_metrics)
    all_metrics.append(train_metrics)
    all_metrics.append(val_metrics)
    all_metrics.append(test_metrics)

    with open(f'../metrics/{experiment}_{count}.pkl', 'wb') as file_pi:
        pickle.dump(all_metrics, file_pi)
    print('saving metrics completed')

9 num_layers: 1 Started at: 2023-05-16 20:58:50.430259
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  736         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 16  2320        ['conv2d[0][0]']                 
                                )      

KeyboardInterrupt: ignored

### 4. Metrics

In [8]:
metrics = []
files = os.listdir(f'../metrics/{experiment}/')

for i in range(len(files) - 1, -1, -1):
    if 'layers' in files[i]:
        del files[i]

files = sorted(files)

for metric_file in files:
    with open(f'../metrics/{experiment}/{metric_file}', 'rb') as file:
        metric_dict = pickle.load(file).__dict__
        metrics.append(metric_dict)

In [9]:
df = pd.DataFrame(metrics)

for i, _ in enumerate(files):
    files[i] = files[i][8:-4]

df.index = files
df

,iou_invalid,iou_valid,iou_land,mean_iou,precision_invalid,precision_valid,precision_land,mean_precision,recall_invalid,recall_valid,recall_land,mean_recall,f1_invalid,f1_valid,f1_land,mean_f1,mean_accuracy
test_1_1,0.855802,0.902306,0.996778,0.918295,0.959200,0.924799,0.998060,0.960687,0.888131,0.973752,0.998713,0.953532,0.922299,0.948645,0.998386,0.956443,0.964831
test_2_1,0.511113,0.722353,0.736162,0.656543,0.528110,0.947211,0.998782,0.824701,0.940762,0.752653,0.736823,0.810080,0.676472,0.838798,0.848034,0.787768,0.789592
test_2_3,0.877207,0.914738,0.997537,0.929828,0.956470,0.941093,0.998234,0.965266,0.913684,0.970294,0.999300,0.961093,0.934588,0.955471,0.998767,0.962942,0.969953
test_3_1,0.898628,0.927567,0.998555,0.941583,0.953436,0.957825,0.999028,0.970096,0.939877,0.967064,0.999526,0.968822,0.946608,0.962423,0.999277,0.969436,0.975138
test_3_2,0.899704,0.928168,0.998078,0.941984,0.950507,0.961136,0.998433,0.970025,0.943925,0.964362,0.999644,0.969310,0.947205,0.962746,0.999038,0.969663,0.975267
test_3_3,0.900827,0.929737,0.998839,0.943134,0.962381,0.953584,0.999259,0.971741,0.933705,0.973807,0.999580,0.969031,0.947826,0.963589,0.999419,0.970278,0.975888
test_4_1,0.908179,0.935095,0.998831,0.947368,0.967426,0.955932,0.999158,0.974172,0.936826,0.977221,0.999673,0.971240,0.951880,0.966459,0.999415,0.972585,0.977758
test_4_2,0.909002,0.935425,0.998509,0.947645,0.961316,0.960634,0.998935,0.973628,0.943514,0.972712,0.999573,0.971933,0.952332,0.966635,0.999254,0.972740,0.977826
test_4_3,0.910239,0.936735,0.998520,0.948498,0.965619,0.958691,0.999065,0.974458,0.940727,0.976134,0.999454,0.972105,0.953010,0.967334,0.999259,0.973201,0.978222
test_5_1,0.904243,0.932276,0.997891,0.944803,0.957190,0.959982,0.998635,0.971936,0.942353,0.969972,0.999254,0.970526,0.949714,0.964951,0.998944,0.971203,0.976531


Note that the indexes containing 'layers' in their name are additional

The table shows that the optimal number of layers is between 3 and 5. The following table therefore only forms the metrics for the test data for trainigs with 3-5 layers, whereby the average value for 3,4 and 5 layers is also calculated.

In [7]:
df_mean = df.iloc[3:11]
mean_3_layers = df.iloc[3:6].mean()
mean_4_layers = df.iloc[6:9].mean()
mean_5_layers = df.iloc[9:11].mean()
df_mean.loc['mean_3_layers'] = mean_3_layers
df_mean.loc['mean_4_layers'] = mean_4_layers
df_mean.loc['mean_5_layers'] = mean_5_layers
df_mean.transpose()

<ipython-input-7-ac548c2e8ab5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mean.loc['mean_3_layers'] = mean_3_layers
<ipython-input-7-ac548c2e8ab5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mean.loc['mean_4_layers'] = mean_4_layers
<ipython-input-7-ac548c2e8ab5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mean.loc['mean_5_layers'] = mean_5_layers


,test_3_1,test_3_2,test_3_3,test_3_4_layers_64_128_256,test_4_1,test_4_2,test_4_3,test_5_1,mean_3_layers,mean_4_layers,mean_5_layers
iou_invalid,0.898628,0.899704,0.900827,0.927203,0.908179,0.909002,0.910239,0.904243,0.899720,0.914795,0.907241
iou_valid,0.927567,0.928168,0.929737,0.947970,0.935095,0.935425,0.936735,0.932276,0.928491,0.939497,0.934505
iou_land,0.998555,0.998078,0.998839,0.999551,0.998831,0.998509,0.998520,0.997891,0.998491,0.998964,0.998206
mean_iou,0.941583,0.941984,0.943134,0.958241,0.947368,0.947645,0.948498,0.944803,0.942234,0.951085,0.946651
precision_invalid,0.953436,0.950507,0.962381,0.973096,0.967426,0.961316,0.965619,0.957190,0.955441,0.967280,0.961404
precision_valid,0.957825,0.961136,0.953584,0.965816,0.955932,0.960634,0.958691,0.959982,0.957515,0.960794,0.959337
precision_land,0.999028,0.998433,0.999259,0.999611,0.999158,0.998935,0.999065,0.998635,0.998907,0.999234,0.998850
mean_precision,0.970096,0.970025,0.971741,0.979508,0.974172,0.973628,0.974458,0.971936,0.970621,0.975769,0.973197
recall_invalid,0.939877,0.943925,0.933705,0.951597,0.936826,0.943514,0.940727,0.942353,0.939169,0.943979,0.941540
recall_valid,0.967064,0.964362,0.973807,0.980881,0.977221,0.972712,0.976134,0.969972,0.968411,0.976938,0.973053


### 5. Conclusion

The previous table shows that the u-nets with a depth of 4 concolutional layers perform best.

It should be noted, however, that this is 2 or 3 trainigs a little too little to take into account statistical variances.

Nevertheless, the mean intersection over union for all 3 runs is worse than that of the best models in the previous experiment. The reason for this could be statistical variance. In addition, transposed convolutional layers were used for the upsampling instead of UpSampling2D layers like in the previous experiments which might have caused the performance decrease.

Therefore 2 additional training runs have been added afterwards. Both configurations use UpSampling2D instead of Conv2DTranspose layers for the upsampling.
Trainig one is named `up_3` as it's U-Net decoder and encoder contain 3 layers, each with 64, 128 and 256 filters. The second training run is named `up_5` as it's U-Net decoder and encoder contain 5 convolutional layers, each with 64, 128, 256, 512 and 1024 filters. In the following table both runs are compared to the best models of the previous table and the best configuration of experiment 7.


In [6]:
metrics = []
all_files = os.listdir(f'../metrics/{experiment}/')
files = []

for i in range(len(all_files) - 1, -1, -1):
    if 'layers' in all_files[i]:
        files.append(all_files[i])

for metric_file in files:
    with open(f'../metrics/{experiment}/{metric_file}', 'rb') as file:
        metric_dict = pickle.load(file).__dict__
        metrics.append(metric_dict)

In [9]:
df = pd.DataFrame(metrics)

df.index = ['test_up_5', 'val_up_5', 'train_up_5', 'test_up_3', 'val_up_3', 'train_up_3']
df.transpose()

,test_up_5,val_up_5,train_up_5,test_up_3,val_up_3,train_up_3
iou_invalid,0.908735,0.907428,0.983374,0.923502,0.958031,0.927203
iou_valid,0.934268,0.933043,0.986902,0.944912,0.965662,0.947970
iou_land,0.998995,0.998924,0.999198,0.999532,0.999598,0.999551
mean_iou,0.947332,0.946465,0.989825,0.955982,0.974430,0.958241
precision_invalid,0.955454,0.961722,0.990473,0.976975,0.985206,0.973096
precision_valid,0.963536,0.957958,0.994313,0.960096,0.977314,0.965816
precision_land,0.999573,0.999455,0.999638,0.999591,0.999646,0.999611
mean_precision,0.972854,0.973045,0.994808,0.978887,0.987389,0.979508
recall_invalid,0.948939,0.941430,0.992764,0.944050,0.972014,0.951597
recall_valid,0.968511,0.972881,0.992505,0.983538,0.987805,0.980881


The best performing model contains three different Conv2D layers with filter sizes of 64, 128, and 256. It uses UpSampling2D instead of TransposedConv2D layers to upsample.

Mean intersection over union for the best performing model:
0.955982